# imports

In [ ]:
import os
path = os.getcwd()
if path[1:5] == "Home":
    %run /Home/siv30/wad005/master/GRB/src/paths/uib.ipynb
    %run /Home/siv30/wad005/master/GRB/src/read_matlab_file.ipynb
    %run /Home/siv30/wad005/master/GRB/src/LC_calibration.ipynb
else: 
    %run /Users/andreas/phys/master/GRB/src/paths/mac.ipynb
    %run /Users/andreas/phys/master/GRB/src/read_matlab_file.ipynb
    %run /Users/andreas/phys/master/GRB/src/LC_calibration.ipynb



In [8]:
import astropy as ap
from stingray import Lightcurve as LC
from stingray import CrossCorrelation
from dateutil import parser

/Users/andreas/miniforge3/lib/python3.9/site-packages/stingray/utils.py:25: UserWarning: pyfftw not installed. Using standard scipy fft
  warnings.warn("pyfftw not installed. Using standard scipy fft")


### General steps to prepare data for xspec
    1) get complete_df with flags removed
    2) shift tus array in dataframe by tshift provided by cross_corrolate(). NOTE: Shift tus_ASIM first manually to around T0. Then CC
    3) use keV_bin() for binning of the energies
    4) change format of dataframe with this command: pd.set_option('display.float_format', '{:.6E}'.format)
    5) export to txt file with command: np.savetxt(r'/Users/andreas/phys/master/GRB/spectral_analysis/GRB190206A_keV_histogram.txt"',df_keV.values, delimiter=',', header='bins_keV,counts,poisson_error_counts,duration_seconds', fmt='%.6E')





## TODO
    -Make this notebook contain only functions so that this notebook can be runned for all ASIM_xspecExtraction notebooks!
    
    -Stingray seems to be a good API for creating EventList object and writing that to a pha and bak file for ASIM
    -Need left most edges in bins_keV for EventList.from_list()
    


## Notes
    -Assuming 
    -bins_keV are the right most edges! 
    -Anders have filtered for keV >=500 
    -Implementation is ok. Proceed to wrapping a function around it.
    --> have a shift variable as input (have to do with lining up the LC from the different spacecrafts)
    
    
## Problems

    

# ToF

In [ ]:
#time of flight from KW to ASIM. Given in seconds
ToF = {"GRB180720":-4.2377,"GRB181222":2.8716, "GRB190206":4.9225, "GRB190305":3.4460,
      "GRB190606":4.0316,"GRB200415":4.1840,"GRB200521":-0.6226,"GRB200716":3.0155,
      "GRB201227":0.7142, "GRB210424":-0.5033, "GRB210619":-2.6183}

In [ ]:
def ToF_correction(grb_id=None):
    ToF_us = ToF[grb_id]*1e6
    df,T0 = complete_df(grb_id=grb_id)
    df["tus"], T0_calibrated = df["tus"] - ToF_us, T0 - timedelta(microseconds=ToF_us) #calibrating 
    return df,T0,T0_calibrated

# Functions for prepping ASIM data for XSPEC

### bin control

In [ ]:
df_bin = pd.read_csv(bin_edges_path + "HED_binedges.txt")
#df_bin.head()

fig,(ax1,ax2) = plt.subplots(1,2,figsize=(15,3))

ax1.plot((df_bin["energy_bin_start_keV"]))
ax2.plot(np.log(df_bin["energy_bin_start_keV"]))
ax1.set_xlabel("bin number")
ax1.set_ylabel("keV bin")
ax2.set_xlabel("bin number")
ax2.set_ylabel("ln(keV)")
plt.show()

start = df_bin["energy_bin_start_keV"].to_numpy()
#start = np.append(0,start)
end = df_bin["energy_bin_stop_keV"].to_numpy()
bins_keV=np.append(start,end[-1])
print(len(bins_keV))
bins_keV

df_hist_check = pd.read_csv(bin_edges_path + "HED_(0.8 ms to 3.2 ms).txt")
print("Num hist counts totoal: ", sum(df_hist_check["counts"].to_numpy()[1:]))
print("bins_keV are the right most edge")

df_hist_check.tail()

### keV bin function

In [2]:
def keV_hist(df=None,time_interval=None,keV_interval=None,keV_bins=None,check_LC=False,convert_seconds=False):
    '''
    
    NOTE: working with [s] resolution for the tus array in dataframe. using float32 --> 4 sig figures for keV
    LED detector sensitive from 20-400 keV
    HED detector sensitive from 200 to >20MeV
    
    input: 
    ---------------------------------------
    note: All times are relative to T0 in the df
    
    df <pd.dataframe>: complete dataframe with no flagged events
    t_start <float>: (optional). Given in [s]. If specified, duration must also be set.
    time_interval <tuple>:  start and end [s] of the time interval for the keV binning.
    keV_interval <tuple> default is None. Meaning the whole spectrum
    keV_bins <np.array> (optional): bins for keV histogram. standard log bins used if None
    convert_seconds <bool>: default True. tus /= 1e6
    return: <pd.dataframe> with the bin_counts and bin_edges as a dataframe
    '''
    df.sort_values(by=['tus'],inplace=True) #first sorting by tus in increasing order
    
    
    #tus handling-------------------------------------------
    tus_arr = df["tus"].to_numpy()
    if convert_seconds == True:
        tus_arr = tus_arr/1e6 #converting to seconds
        
    if time_interval == 'max':
        idx_start,idx_end = 0,-1 
    elif type(time_interval) == tuple:
        idx_start, idx_end = find_nearest(tus_arr,time_interval[0]), find_nearest(tus_arr,time_interval[1])+1
    else:
        return(print('Time interval must be specified'))

    tus_arr = tus_arr[idx_start:idx_end] #selecting the interval for tus
    
    #keV handling-------------------------------------------
    keV_arr = df["keV"].to_numpy(dtype=np.float32)
    keV_arr = keV_arr[idx_start:idx_end] #selecting keV's on the given tus interval
    
    if keV_interval==None:
         return print("keV bounds must be specified")
    elif keV_interval == "max":
        keV_lower, keV_upper = 1e1, 1e5
        print("Computing whole spectrum (10-1e5 keV)")
    else: 
        keV_lower,keV_upper = keV_interval[0],keV_interval[1] #selecting the bounds for the keV's

    ii = []
    for i,keV in enumerate(keV_arr): #finding
        if keV_lower<=keV<=keV_upper:
            ii.append(i)
    
    #Selecting the triggers that are within the time & energy bounds
    tus_keV_bound = tus_arr[ii]
    keV_bound = keV_arr[ii]
    print("n triggers in time interval ",len(keV_arr))
    print("n triggers are within bounds ", len(ii))
    
    #histogramming-----------------------------------------
    #Just using the left bins + the last upper for binning
    if keV_bins == None:
        df_bin = pd.read_csv(bin_edges_path + "HED_binedges.txt")
        print("keV bins are set for HED")
        keV_bin_start = df_bin["energy_bin_start_keV"].to_numpy()
        #start = np.append(0,start)
        keV_bin_end = df_bin["energy_bin_stop_keV"].to_numpy()
        keV_bins = np.append(keV_bin_start,keV_bin_end[-1]) #just appending the last bin

    
    hist_keV, edges_keV = np.histogram(keV_bound,bins=keV_bins)
    
    
    #Check this block of code for errors!
    poisson_arr = np.sqrt(hist_keV) #handle this one. might be some negative numbers in it?
    if hist_keV[0] == 0:
        hist_keV = np.append(0,hist_keV)
        poisson_arr = np.append(0,poisson_arr)
    df_keV = pd.DataFrame({"bins_keV":edges_keV, "counts":hist_keV, 
                           "poisson_error_counts":poisson_arr}) #ERROR different lengths!
    print("Energies binned!")
    
    if check_LC != False:
        return df_keV, tus_keV_bound
    else:
        return df_keV

### Cross Corrolate function

In [ ]:
def shiftTus(tus=None, tlagg=None):
    """ Shift tus by adding tlagg to tus-array
    input:
    tus <np.array> must be given in us!
    tlagg <float> must be gievn in us!
    
    return:
    tusShift <np.array> given in us """
    
    tusShift = tus + tlagg
    return tusShift

### General steps to prepare data for xspec
    1) get complete_df with flags removed
    2) shift tus array in dataframe by tshift provided by cross_corrolate(). NOTE: ADDED to tus_ASIM
    3) use keV_bin() for binning of the energies
    4) change format of dataframe with this command: pd.set_option('display.float_format', '{:.6E}'.format)
    5) export to txt file with command: np.savetxt(r'/Users/andreas/phys/master/GRB/spectral_analysis/GRB190206A_keV_histogram.txt"',df_keV.values, delimiter=',', header='bins_keV,counts,poisson_error_counts,duration_seconds', fmt='%.6E')



from astropy.table import Table
df = pd.read_csv("/Users/andreas/phys/master/GRB/spectral_analysis/keV_histograms/GRB180720")
t = Table.from_pandas(df)
t.write('GRB180720_new.fits')